# IsicApi overview (from isicarchive package)

## Installation
To use the IsicApi class, please first install the isicarchive
package from github, which is more frequently updated for now:
~~~~
pip install --upgrade git+https://github.com/neuroelf/isicarchive
~~~~

Or for a more stable release, you can install from PyPi using:
~~~~
pip install --upgrade isicarchive
~~~~

In [1]:
# Make sure the latest isicarchive package is installed!
import sys
!{sys.executable} -m pip install --upgrade git+https://github.com/neuroelf/isicarchive

  Cloning https://github.com/neuroelf/isicarchive to c:\users\weberj3\appdata\local\temp\pip-req-build-4hdg2irn
  Stored in directory: C:\Users\weberj3\AppData\Local\Temp\pip-ephem-wheel-cache-mue26d57\wheels\ba\cf\32\48caf3dba09770da83411064c00c38010306f48fd3bcdb784a
Successfully built isicarchive
  Found existing installation: isicarchive 0.4.0
    Can't uninstall 'isicarchive'. No files were found to uninstall.


  Running command git clone -q https://github.com/neuroelf/isicarchive 'C:\Users\weberj3\AppData\Local\Temp\pip-req-build-4hdg2irn'


## Import and use the ```IsicApi``` object from the isicarchive module

### ***Please change the username and cache_folder!***
And please note that depending on which user is logged into the
database, the responses from the (web-based) API differ: for
instance, the number of datasets, images, and studies you have
access to depends on the settings on these datasets, images, and
studies.

If you wish to work with the IsicApi object both with and without
user permissions, please use two **separate** cache folders, for
otherwise the image caching routine will be thrown off by the
different number of images you have access to!

The image cache file will be written into
```<CACHE_FOLDER>/0/imcache_000000000000000000000000.json.gz```
and for the full database (being authenticated with admin privileges)
takes about 3.5MByte at this time (~ 100MB uncompressed).

You may want to periodically re-create this cache file if you use
the database without user authentication, since the order in which
the image are received is based on their objectId in the database,
meaning that if images that were uploaded but only recently made
public, the cache will be "out of sync" and needs to be recreated.

In [2]:
# Import isicapi
from isicarchive import IsicApi, func

# Set username and password (to None!)
username = None
password = None
cache_folder = 'C:\\Users\\weberj3\\Documents\\ISIC'
#cache_folder = '/Data/MSKCC/ISIC'

# Instantiate API, will ask for password using getpass.getpass(...)
api = IsicApi(username, password, cache_folder=cache_folder)

In [3]:
# load cached images
api.cache_images()

In [4]:
# pretty-print api object
api

isicarchive.api.IsicApi({
    'base_url': 'https://isic-archive.com/api/v1',
    'username': None,
    'cache_folder': 'C:\Users\weberj3\Documents\ISIC',
    'cached_images': { ... dict with 23906 fields},
    'loaded_datasets': { ... dict with 13 fields},
    'loaded_studies': { ... dict with 2 fields},
    'obj_annotations': { ... dict with 0 fields},
    'obj_datasets': { ... dict with 0 fields},
    'obj_images': { ... dict with 0 fields},
    'obj_studies': { ... dict with 0 fields},})

## Example usage to retrieve a study and an image from the study
Studies (as well as datasets and images) can be retrieved either
using their objectId (```_id``` field in the web-based API), or
using their name. Please note, some images in studies may come
from datasets that are not publicly available. If you are not
logged into the API with the necessary privileges, you will only
be able to access the portion of images you have access to.

In [5]:
# print available studies
api.list_studies()

List of studies available at https://isic-archive.com:
 * ISBI 2016: 100 Lesion Classification (id=573f11119fc3c132505c0ee7)
     -  (description)
     - 800 annotations (0 loaded)
     - 0 features
     - 100 images (100 loaded)
     - 3 questions


C:\Users\weberj3\Documents\git\isicarchive\isicarchive\study.py:458: UserWarning: 107 images could not be loaded.
  len(to_load) - len(image_info)))


 * ISIC Annotation Study - All Features (id=5a32cde91165975cf58a469c)
     -  (description)
     - 840 annotations (0 loaded)
     - 85 features
     - 140 images (33 loaded)
     - 2 questions


In [6]:
# Retrieve a study
study = api.study('ISIC Annotation Study - All Features')

# load annotations (will only work if logged in!)
study.load_annotations()

# print some information about the study
study

Loading annotations: |########################################################################| 100.0% 


isicarchive.study.Study({
    'id': '5a32cde91165975cf58a469c',
    'name': 'ISIC Annotation Study - All Features',
    'description': '',
    'annotations': [ ... list with 840 items],
    'features': [ ... list with 85 items],
    'images': [ ... list with 140 items],
    'questions': [ ... list with 2 items],
    'users': [ ... list with 6 items],})

In [7]:
# Retrieve an image for this study
image = api.image(study.images[0])

# Load the image data (from cache or online)
image.load_imagedata()

# And load the superpixels, and parse into coordinates map
image.load_superpixels()
image.map_superpixels()

# print information
image

isicarchive.image.Image({
    'id': '54e755f8bae47850e86ce012',
    'name': 'ISIC_0001115',
    'dataset_id': '5a2ecc5e1165975c945942a4',
    'dataset_name': 'UDA-2',
    'meta_acquisition': '{'image_type': 'dermoscopic', 'pixelsX': 1936, 'pixelsY': 1936}',
    'meta_clinical': '{'age_approx', 'anatom_site_general', 'benign_malignant', 'diagnosis', 'diagnosis_confirm_type', 'melanocytic', 'sex'}',
    'meta_clinical_benign_malignant': 'benign',
    'meta_clinical_diagnosis': 'angioma',
    'meta_clinical_diagnosis_confirm_type': 'histopathology',
    'meta_clinical_melanocytic': False,
    'notes_reviewed': '{'accepted': True, 'time': '2015-02-20T15:47:42.085000+00:00', 'userId': '5450e996bae47865794e4d0d'}',
    'superpixels_max': 1020,
    'superpixels_shape': (1936, 1936),})

In [8]:
# show in notebook
image.show_in_notebook(max_size=400)

C:\Users\weberj3\Documents\git\isicarchive\isicarchive\image.py:401: UserWarning: Problem producing image for display: The 'value' trait of an Image instance must be a bytes object, but a value of 'C:\\Users\\weberj3\\Documents\\ISIC\\2\\image_54e755f8bae47850e86ce012_ISIC_0001115.jpg' <class 'str'> was specified.
  warnings.warn('Problem producing image for display: ' + str(e))


In [9]:
# Retrieve an annotation
annotation = api.annotation(study.annotations[0])

# print information about the annotation
annotation

isicarchive.annotation.Annotation({
    'id': '5a32cde91165975cf58a469e',
    'study_id': '5a32cde91165975cf58a469c',
    'study_name': 'ISIC Annotation Study - All Features',
    'image_id': '54e755f8bae47850e86ce012',
    'image_name': 'ISIC_0001115',
    'user_id': '578e64b09fc3c10d6fd12e4f',
    'user_name': 'User 6YB2',
    'user_lastname': None,
    'features': '{'Globules / Clods : Lacunae : Red', 'Shiny white structures : Shiny white blotches and strands'}',})

In [10]:
# Show the first feature in cyan, 70% opacity
annotation.show_in_notebook(features={'Globules / Clods : Lacunae : Red': [[0,255,255], 0.7]})

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe1"\x18Exif\x00\x00MM\x00*\x…

In [11]:
# and get some information on the dataset of the image
dataset = api.dataset(image.dataset)
dataset.load_images()
dataset

isicarchive.dataset.Dataset({
    'id': '5a2ecc5e1165975c945942a4',
    'name': 'UDA-2',
    'description': 'Moles and melanomas.
Biopsy-confirmed melanocytic lesions. Both malignant and benign lesions are included.',
    'access_level': 0,
    'license': 'CC-0',
    'owner': 'Anonymous',
    'updated': '2015-03-06T14:54:30.984000+00:00',
    'images': [ ... list with 60 items],
    'images_for_review': [ ... list with 0 items],})

In [12]:
# show several annotations for several images
study.show_annotations(images=['ISIC_0001115', 'ISIC_0001129', 'ISIC_0001130', 'ISIC_0001149'])